In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
# Generate from Folders
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
#test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
#train_generator = train_datagen.flow_from_directory(
#        trainFolder,
#        batch_size=32,
#        class_mode='categorical')
#validation_generator = test_datagen.flow_from_directory(
#        valFolder,
#        batch_size=32,
#        class_mode='categorical')

# DATA AUGMENTATION via Generator

batch_size = 32
train_input_shape = (224, 224, 3)
n_classes = 10
print(n_classes)

# is preprocessing line required?
shared = "drive/Shareddrives/Project F Folder/"
trainFolder = shared + "ImageGenFolder4/train/"
valFolder = shared + "ImageGenFolder4/val/"

train_datagen = ImageDataGenerator(
                                   rescale=1./255.,
                                   rotation_range=30,
                                   zoom_range=0.5,
                                   #horizontal_flip=True,
                                   #vertical_flip=True,
                                  )

val_datagen = ImageDataGenerator(
                                   rescale=1./255.,
                                   #horizontal_flip=True,
                                   #vertical_flip=True,
                                  )

train_generator = train_datagen.flow_from_directory(directory=trainFolder,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                   )

valid_generator = val_datagen.flow_from_directory(directory=valFolder,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                   )

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID)

C:\Users\Radek Pudelko\AppData\Roaming\Python\Python37\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


10
Found 1600 images belonging to 10 classes.
Found 400 images belonging to 10 classes.
Total number of batches = 50 and 12


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet101
#from tensorflow_addons.metrics import F1Score


class F1_Score(tf.keras.metrics.Metric):

    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1 = self.add_weight(name='f1', initializer='zeros')
        self.precision_fn = tf.keras.metrics.Precision(thresholds=0.5)
        self.recall_fn = tf.keras.metrics.Recall(thresholds=0.5)

    def update_state(self, y_true, y_pred, sample_weight=None):
        p = self.precision_fn(y_true, y_pred)
        r = self.recall_fn(y_true, y_pred)
        # since f1 is a variable, we use assign
        self.f1.assign(2 * ((p * r) / (p + r + 1e-6)))

    def result(self):
        return self.f1

    def reset_state(self):
        # we also need to reset the state of the precision and recall objects
        self.precision_fn.reset_states()
        self.recall_fn.reset_states()
        self.f1.assign(0)



Metrics = ["acc","Precision", "Recall", F1_Score()]


def FCN3(base, dense1, dense2):
    MY_SIZE = 224
    IMG_SIZE = (MY_SIZE,MY_SIZE)
    IMG_SHAPE = IMG_SIZE + (3,)

    model = tf.keras.Sequential([
        base.layers[0],
        base.layers[1],
        
        layers.Dense(dense1, activation="relu"),
        layers.Dropout(.5),
        layers.BatchNormalization(),
        
        layers.Dense(dense2, activation="relu"),
        layers.Dropout(.5),
        layers.BatchNormalization(),
        
        layers.Dense(n_classes, activation = "softmax"),   
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
        loss='categorical_crossentropy',
        metrics=Metrics)
    return model


In [ ]:
patience = 6
earlystopping = tf.keras.callbacks.EarlyStopping(monitor="val_f1_score", patience=patience,restore_best_weights=False, mode="max", verbose=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_f1_score', factor=0.1,patience=3, 
                              verbose=1, mode='max')



In [ ]:
models = []
histories = []
allTrials = []
trials = []
results = []

dense1 = list(range(128,512,64))
dense2 = list(range(32,256,32))

combos = []
for d1 in dense1:
    for d2 in dense2:
        combos.append([d1,d2])
print(len(combos))


42


In [ ]:
import os
# Load the results from previous runs 
resultsFile = shared + "myResults3.csv"
if os.path.exists(resultsFile):
    with open(resultsFile, "r") as out:
        raw = out.readlines()
        for line in raw[1:]:
            line = line.split(",")
            combo = [int(line[0]), int(line[1])]
            if combo in combos:
                print(combo)
                combos.pop(combos.index(combo))
            allTrials.append(combo)
            result = [float(line[2]), float(line[3]), float(line[4]), float(line[5]), float(line[6])]
            results.append(result)
print(len(combos))

[448, 224]
41


In [ ]:
import random


for i in range(5):
    
    randI = random.randint(0, len(combos)-1)
    combo = combos.pop(randI)
    trials.append(combo)
    print(i, combo[0], combo[1])
    base = tf.keras.models.load_model(shared + 'myBaseModel.h5', custom_objects={"F1_Score": F1_Score})
    myModel = FCN3(base, combo[0], combo[1])
    modelPath = shared + "models/model_" + str(combo[0]) + "_" + str(combo[1]) + ".h5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        modelPath, monitor='val_f1_score', verbose=1, save_best_only=True,
        save_weights_only=False, mode='max', save_freq='epoch',
        options=None)
    history = myModel.fit(train_generator, validation_data = valid_generator, epochs=100, verbose = 1,
                      callbacks=[earlystopping, reduce_lr, checkpoint])
    histories.append(history)
    models.append(myModel)
    print("\n\n\n\n#################################################")
    print("#################################################")
    print("#################################################\n\n\n\n")
    

0 192 224
Epoch 1/100
50/50 [==============================] - 36s 537ms/step - loss: 2.0132 - acc: 0.3281 - precision: 0.5336 - recall: 0.1587 - f1_score: 0.2447 - val_loss: 0.9602 - val_acc: 0.7275 - val_precision: 0.9000 - val_recall: 0.5625 - val_f1_score: 0.6923

Epoch 00001: val_f1_score improved from -inf to 0.69231, saving model to models\model_192_224.h5


c:\users\radek pudelko\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
50/50 [==============================] - 26s 522ms/step - loss: 1.2581 - acc: 0.6000 - precision: 0.7888 - recall: 0.3969 - f1_score: 0.5281 - val_loss: 0.5610 - val_acc: 0.8525 - val_precision: 0.9177 - val_recall: 0.7525 - val_f1_score: 0.8269

Epoch 00002: val_f1_score improved from 0.69231 to 0.82692, saving model to models\model_192_224.h5
Epoch 3/100
50/50 [==============================] - 25s 505ms/step - loss: 0.9864 - acc: 0.6981 - precision: 0.8538 - recall: 0.5400 - f1_score: 0.6616 - val_loss: 0.7740 - val_acc: 0.7775 - val_precision: 0.8534 - val_recall: 0.7275 - val_f1_score: 0.7854

Epoch 00003: val_f1_score did not improve from 0.82692
Epoch 4/100
50/50 [==============================] - 25s 501ms/step - loss: 0.8384 - acc: 0.7437 - precision: 0.8758 - recall: 0.6081 - f1_score: 0.7178 - val_loss: 0.7240 - val_acc: 0.7500 - val_precision: 0.8593 - val_recall: 0.7025 - val_f1_score: 0.7730

Epoch 00004: val_f1_score did not improve from 0.82692
Epoch 5/100
5

Epoch 10/100
50/50 [==============================] - 25s 505ms/step - loss: 0.3761 - acc: 0.9075 - precision: 0.9551 - recall: 0.8500 - f1_score: 0.8995 - val_loss: 0.4294 - val_acc: 0.8775 - val_precision: 0.9272 - val_recall: 0.8275 - val_f1_score: 0.8745

Epoch 00010: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.

Epoch 00010: val_f1_score did not improve from 0.87895
Epoch 11/100
50/50 [==============================] - 25s 504ms/step - loss: 0.3433 - acc: 0.9175 - precision: 0.9544 - recall: 0.8506 - f1_score: 0.8995 - val_loss: 0.4316 - val_acc: 0.8775 - val_precision: 0.9274 - val_recall: 0.8300 - val_f1_score: 0.8760

Epoch 00011: val_f1_score did not improve from 0.87895
Epoch 12/100
50/50 [==============================] - 26s 515ms/step - loss: 0.3597 - acc: 0.9162 - precision: 0.9492 - recall: 0.8525 - f1_score: 0.8983 - val_loss: 0.4316 - val_acc: 0.8775 - val_precision: 0.9274 - val_recall: 0.8300 - val_f1_score: 0.8760

Epoch 00012: val_f1_score did

KeyboardInterrupt: 

In [ ]:
hResults = []
resI = -1 - patience
for i in range(len(histories)):
    hRes = []
    hRes.append(histories[i].history["val_loss"][resI])
    hRes.append(histories[i].history["val_acc"][resI])
    hRes.append(histories[i].history["val_precision"][resI])
    hRes.append(histories[i].history["val_recall"][resI])
    hRes.append(histories[i].history["val_f1_score"][resI])
    print(hRes)
    hResults.append(hRes)
print()

In [ ]:
allTrials.extend(trials)
results.extend(hResults)
print(len(trials))
print(len(results))
for i in range(len(results)):
    print(i, allTrials[i], results[i][-1])


In [ ]:
with open(resultsFile, "w") as out:
    out.write("dense1,dense2,loss,acc,prec,recall,f1\n")
    for i in range(len(results)):
        line = str(allTrials[i][0]) + "," + str(allTrials[i][1]) + ","
        line += str(results[i][0])
        for res in results[i][1:5]:
            print(i, res)
            line += "," + str(res)
        out.write(line + "\n")